In [1]:
!pip install influxdb-client

   ---------------------------------------- 0.0/746.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/746.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/746.3 kB ? eta -:--:--
   ---------------------------- ----------- 524.3/746.3 kB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 746.3/746.3 kB 2.0 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [reactivex]
   ---------------------------------------- 0/2 [reactivex]
   ---------------------------------------- 0/2 [reactivex]
   ---------------------------------------- 0/2 [reactivex]
   ---------------------------------------- 0/2 [reactivex]
   ---------------------------------------- 0/2 [reactivex]
   ---------------------------------------- 0/2 [reactivex]
   ---------------------------------------- 0/2 [reactivex]
   ---------------------------------------- 0/2 [reactivex]
   ---------------------------------------- 0/2 [reactivex]
   -----------------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from influxdb_client import InfluxDBClient
from influxdb_client.client.query_api import QueryApi

In [4]:
!pip install pandas

  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl (11.0 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)

   ---------------------------------------- 0/2 [pytz]
   ---------------------------------------- 0/2 [pytz]
   ---------------------------------------- 0/2 [pytz]
   ---------------------------------------- 0/2 [pytz]
   ---------------------------------------- 0/2 [pytz]
   ---------------------------------------- 0/2 [pytz]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   --------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

#### **_InfluxDB connection and Client Code_**

In [6]:
INFLUXDB_URL = "https://us-east-1-1.aws.cloud2.influxdata.com"
INFLUXDB_TOKEN = ""
ORG = "MyAI/ML_Workspace"
BUCKET = "Sample-SensorData"

client = InfluxDBClient(
    url=INFLUXDB_URL,
    token=INFLUXDB_TOKEN,
    org=ORG
)

query_api: QueryApi = client.query_api()

query = f'''
from(bucket: "{BUCKET}")
  |> range(start: -1h)
  |> filter(fn: (r) => r._measurement == "field_data")
'''

tables = query_api.query(query, org=ORG)

records = []

for table in tables:
    for record in table.records:
        records.append({
            "time": record.get_time(),
            "field": record.get_field(),
            "value": record.get_value(),
            "location": record.values.get("location"),
            "crop": record.values.get("crop"),
            "sensor_type": record.values.get("sensor_type")
        })

df = pd.DataFrame(records)

print("Fetched Sensor Data:")
print(df.head())

client.close()

Fetched Sensor Data:
                              time             field    value location   crop  \
0 2025-12-24 06:28:12.621214+00:00  humidity_percent     52.0  Field_A  Wheat   
1 2025-12-24 06:29:14.724708+00:00  humidity_percent     51.0  Field_A  Wheat   
2 2025-12-24 06:30:03.073791+00:00  humidity_percent     74.0  Field_A  Wheat   
3 2025-12-24 06:28:12.621214+00:00         light_lux  26000.0  Field_A  Wheat   
4 2025-12-24 06:29:14.724708+00:00         light_lux  29000.0  Field_A  Wheat   

  sensor_type  
0   EnvSensor  
1   EnvSensor  
2   EnvSensor  
3   EnvSensor  
4   EnvSensor  


#### **_Fetched Data into Summaries**_

In [11]:
df.head()

,time,field,value,location,crop,sensor_type
0,2025-12-24 06:28:12.621214+00:00,humidity_percent,52.0,Field_A,Wheat,EnvSensor
1,2025-12-24 06:29:14.724708+00:00,humidity_percent,51.0,Field_A,Wheat,EnvSensor
2,2025-12-24 06:30:03.073791+00:00,humidity_percent,74.0,Field_A,Wheat,EnvSensor
3,2025-12-24 06:28:12.621214+00:00,light_lux,26000.0,Field_A,Wheat,EnvSensor
4,2025-12-24 06:29:14.724708+00:00,light_lux,29000.0,Field_A,Wheat,EnvSensor


In [8]:
df.describe()

,value
count,15.000000
mean,3958.246667
std,9628.827603
min,6.600000
25%,24.250000
50%,33.100000
75%,63.000000
max,29000.000000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   time         15 non-null     datetime64[ns, UTC]
 1   field        15 non-null     object             
 2   value        15 non-null     float64            
 3   location     15 non-null     object             
 4   crop         15 non-null     object             
 5   sensor_type  15 non-null     object             
dtypes: datetime64[ns, UTC](1), float64(1), object(4)
memory usage: 852.0+ bytes


In [12]:
def generate_sensor_summary(df, location, crop):
    summary_lines = []

    summary_lines.append(
        f"Sensor Summary for {crop} crop at {location} (recent readings):\n"
    )

    grouped = df.groupby("field")["value"]

    for field_name, values in grouped:
        avg = values.mean()
        min_val = values.min()
        max_val = values.max()

        if field_name == "humidity_percent":
            summary_lines.append(
                f"- Humidity ranged between {min_val:.0f}% and {max_val:.0f}%, "
                f"with an average of {avg:.1f}%."
            )

        elif field_name == "light_lux":
            summary_lines.append(
                f"- Light intensity varied from {min_val:.0f} lux to {max_val:.0f} lux, "
                f"averaging around {avg:.0f} lux."
            )

        elif field_name == "temperature_c":
            summary_lines.append(
                f"- Temperature ranged between {min_val:.1f}°C and {max_val:.1f}°C, "
                f"with an average of {avg:.1f}°C."
            )

        elif field_name == "soil_moisture_percent":
            summary_lines.append(
                f"- Soil moisture levels were between {min_val:.0f}% and {max_val:.0f}%, "
                f"averaging {avg:.1f}%."
            )

        elif field_name == "soil_ph":
            summary_lines.append(
                f"- Soil pH remained stable between {min_val:.2f} and {max_val:.2f}, "
                f"with an average of {avg:.2f}."
            )

        else:
            summary_lines.append(
                f"- {field_name} ranged between {min_val:.2f} and {max_val:.2f} "
                f"(avg: {avg:.2f})."
            )

    return "\n".join(summary_lines)


In [13]:
location = df["location"].iloc[0]
crop = df["crop"].iloc[0]

sensor_summary = generate_sensor_summary(df, location, crop)
print(sensor_summary)


Sensor Summary for Wheat crop at Field_A (recent readings):

- Humidity ranged between 51% and 74%, with an average of 59.0%.
- Light intensity varied from 4000 lux to 29000 lux, averaging around 19667 lux.
- Soil moisture levels were between 26% and 36%, averaging 29.3%.
- Soil pH remained stable between 6.60 and 7.10, with an average of 6.77.
- Temperature ranged between 22.5°C and 33.1°C, with an average of 29.5°C.
